# 1.Packages

In [1]:
import time
import cv2
import os
import numpy as np
import scipy.fftpack as ffpack #Fourier transform API
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import mean_squared_error as compare_mse
from skimage.restoration import denoise_wavelet

# 2.MSE and SSIM

In [2]:
def MseAndSsim(OriginalImage,DenoiseImage):
    mse=compare_mse(DenoiseImage,OriginalImage)#mse
    ssim=compare_ssim(DenoiseImage,OriginalImage,multichannel=True)#ssim
    return mse,ssim

# 3.Algorithm

In [3]:
#Denoise function
def Denoise(OriginalImage,NoiseImage):
    #Non-local mean method
    NoiseImage=cv2.fastNlMeansDenoisingColored(NoiseImage,None,10,10,7,21)
    #for noise50
    #NoiseImage=cv2.fastNlMeansDenoisingColored(NoiseImage,None,20,20,7,21)
    
    #chanel split:B、G、R
    B,G,R=cv2.split(NoiseImage)
    
    # B part do Fourier
    B_fft=ffpack.fft2(B)
    mean_b=np.abs(B_fft).mean()
    #High-frequency threshold processing
    img_high=np.abs(B_fft)>7e9
    #Greater than this value is judged to be 0
    B_fft[img_high]=0
    #Fourier inverse transform
    B_res=ffpack.ifft2(B_fft)
    #Take the real part
    B_res=np.real(B_res)
    #The value ranges from 0 to 255
    B_res=np.uint8(B_res)
    
    # G part
    G_fft=ffpack.fft2(G)
    mean_g=np.abs(G_fft).mean()
    #High-frequency threshold processing
    img_high=np.abs(G_fft)>7e9
    #Greater than this value is judged to be 0
    G_fft[img_high]=0
    #Fourier inverse transform
    G_res=ffpack.ifft2(G_fft)
    #Take the real part
    G_res=np.real(G_res)
    #The value ranges from 0 to 255
    G_res=np.uint8(G_res)
    
    # R part
    R_fft=ffpack.fft2(R)
    mean_r=np.abs(R_fft).mean()
    #High-frequency threshold processing
    img_high=np.abs(R_fft)>7e9
    #Greater than this value is judged to be 0
    R_fft[img_high]=0
    #Fourier inverse transform
    R_res=ffpack.ifft2(R_fft)
    #Take the real part
    R_res=np.real(R_res)
    #The value ranges from 0 to 255
    R_res=np.uint8(R_res)
    
    #B,G，R do median filter
    B_res1=cv2.medianBlur(B_res,5)
    G_res1=cv2.medianBlur(G_res,5)
    R_res1=cv2.medianBlur(R_res,5)
   
    #merge
    BGR=cv2.merge([B_res1,G_res1,R_res1])
    #mse，ssim
    my_mse,my_ssim=MseAndSsim(BGR,OriginalImage)
    
    #return value
    return my_mse,my_ssim,BGR

In [5]:
#Traversal folder
#path_original = r'C:\Users\Lenovo\Desktop\My Algorithm\images\original'
#path_noise = r'C:\Users\Lenovo\Desktop\My Algorithm\images\noisy10'
path_original = './images/./original/'
path_noise ='./images/./noisy10/'
for fname in os.listdir(path_noise):
    img = os.path.join(path_original, fname)
    img1 = os.path.join(path_noise, fname)
    original_img=cv2.imread(img)
    img_noise = cv2.imread(img1)
    rows, cols = original_img.shape[:2] 
    img_noise = cv2.resize(img_noise,(cols,rows)) 
    
    #start denoising
    f_mse,f_ssim,img_f_denoise = Denoise(original_img, img_noise) 
    
    #show images and print our result
    image=np.hstack([original_img,img_noise,img_f_denoise])
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow("image", image)
    cv2.waitKey(0)
    print(img1)
    print('FT--SSIM:{}  MSE：{}'.format(f_ssim, f_mse))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_164204\1667765818.py:3: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim=compare_ssim(DenoiseImage,OriginalImage,multichannel=True)#计算ssim


./images/./noisy10/0000.png
FT--SSIM:0.9542044847139534  MSE：17.508185396035863
./images/./noisy10/0001.png
FT--SSIM:0.5834606672657642  MSE：274.6274031040386
./images/./noisy10/0002.png
FT--SSIM:0.8235744303111643  MSE：48.76062331202518
./images/./noisy10/0003.png
FT--SSIM:0.6357304814239595  MSE：201.9806585881352
./images/./noisy10/0004.png
FT--SSIM:0.656515746641177  MSE：228.97689350025797


KeyboardInterrupt: 

# 4.Algorithm comparison

In [6]:
def MedianDenoise(o_img, noise_img):
    image_media=cv2.medianBlur(noise_img,5) 
    mse,ssim=MseAndSsim(image_media,o_img)
    return mse,ssim,image_media 
def WaveletDenoise(o_img, noise_img):
    image_wavelet=denoise_wavelet(noise_img,wavelet='db1',mode='soft',wavelet_levels=3,convert2ycbcr=True,multichannel=True,method='BayesShrink')
    image_wavelet=np.uint8(image_wavelet*256)
    mse,ssim=MseAndSsim(image_media,o_img)
    return mse,ssim,image_wavelet 
def MeanDenoise(o_img, noise_img):
    image_means=cv2.blur(noise_img,(1,5))
    mse,ssim=MseAndSsim(image_media,o_img)
    return mse,ssim,image_means 

In [7]:
#load the data
o_img=cv2.imread('./images/./original/./0000.png')
#noise_img=cv2.imread('./images/./noisy10/./0000.png')
#noise_img=cv2.imread('./images/./noisy25/./0004.png')
noise_img=cv2.imread('./images/./noisy10/./0000.png')
rows, cols = o_img.shape[:2] #获取orinal的高度、宽度
noise_img = cv2.resize(noise_img,(cols,rows)) #将噪声图转换为原图同样尺寸

In [8]:
#computing
median_mse,median_ssim,image_media=MedianDenoise(o_img,noise_img)
w_mse,w_ssim,image_w=WaveletDenoise(o_img,noise_img)
mean_mse,mean_ssim,image_mean=MeanDenoise(o_img,noise_img)
my_mse,my_ssim,image_my=Denoise(o_img,noise_img)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_164204\1667765818.py:3: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim=compare_ssim(DenoiseImage,OriginalImage,multichannel=True)#计算ssim
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_164204\2494223129.py:6: FutureWarning: `multichannel` is a deprecated argument name for `denoise_wavelet`. It will be removed in version 1.0. Please use `channel_axis` instead.
  image_wavelet=denoise_wavelet(noise_img,wavelet='db1',mode='soft',wavelet_levels=3,convert2ycbcr=True,multichannel=True,method='BayesShrink')


In [9]:
#show imamges
cv2.imshow("mean filter", image_mean)
cv2.imshow('median filter',image_media)
cv2.imshow('wavelet',image_w)
cv2.imshow('my',image_my)
cv2.waitKey(0)

-1

In [10]:
#print result
print(' means filter MSE:{}, SSIM:{}'.format(mean_mse,mean_ssim))
print(' median filter MSE:{}, SSIM:{}'.format(median_mse,median_ssim))
print(' wavelet MSE:{}, waveletSSIM:{}'.format(w_mse,w_ssim))
print(' My MSE:{}, MySSIM:{}'.format(my_mse,my_ssim))

 means filter MSE:15.236602526322152, SSIM:0.9360518118953824
 median filter MSE:15.236602526322152, SSIM:0.9360518118953824
 wavelet MSE:15.236602526322152, waveletSSIM:0.9360518118953824
 My MSE:17.508185396035863, MySSIM:0.9542044847139534
